In [ ]:
import mnist
from torch import Tensor
import torch

# all available training data
print(Tensor(mnist.train_images()).shape)

In [ ]:
# load in data
mnist_training_data = Tensor(mnist.train_images()[:2000])
normalized_mnist_training_data = (mnist_training_data) / 255
mnist_label_data = Tensor(mnist.train_labels()[:2000])

In [ ]:
# load in model
from lvmogp_svi import LVMOGP_SVI
# define hyper-parameters
n_X = mnist_training_data.shape[0]
n_C = int(mnist_training_data.shape[-2]*mnist_training_data.shape[-1]) # also data_dim
n_total = n_X * n_C
index_dim = 2
latent_dim = 20
n_inducing_C = 30
n_inducing_X = 50
pca = True
Y_train = mnist_training_data.reshape(-1)

forbidden_pairs = [(2, 18), (2, 79), (7, 183), (8, 201), (5,23), (6, 99), (0,3), (7, 139), (8, 143), (5, 34)]

# specify model
my_model = LVMOGP_SVI(n_X, n_C, index_dim, latent_dim, n_inducing_C, n_inducing_X, Y_train.reshape(n_X, -1), pca=pca)

model_path = '/Users/jiangxiaoyu/Desktop/All Projects/GPLVM_project_code/models/model_weights2.pth'
state_dict = torch.load(model_path)
my_model.load_state_dict(state_dict)

my_model.eval()

In [ ]:
length_scale = [param.detach() for name, param in my_model.named_parameters() if name.startswith('covar_module_X')]
inverse_length_scale = (1 / length_scale[-1])[0]
print(inverse_length_scale)

In [ ]:
values, indices = torch.sort(inverse_length_scale, descending=True)
print(indices)

In [ ]:
latents_2d = my_model.X.q_mu.detach()[:, indices[:2]]

In [ ]:
print(latents_2d.shape)
print(mnist_label_data.shape)

In [ ]:
import torch
import matplotlib.pyplot as plt

unique_labels = torch.unique(mnist_label_data)

plt.figure(figsize=(10, 8))

for label in unique_labels:
    mask = mnist_label_data == label
    subset = latents_2d[mask]
    
    plt.scatter(subset[:, 0], subset[:, 1], label=str(label.item()))

plt.legend()
plt.show()

In [ ]:
# V1
from locale import normalize
import mnist
import numpy as np
import torch
from torch import Tensor
from lvmogp_svi import LVMOGP_SVI
from gaussian_likelihood import GaussianLikelihood
from variational_elbo import VariationalELBO
from tqdm import trange
from torch.optim.lr_scheduler import StepLR
from util_functions import remove_forbidden_pairs, inhomogeneous_index_of_batch_Y, proper_sample_index_X_and_C, random_sample_index_X_and_C, tidily_sample_index_X_and_C

# load in mnist data (scale 0-255)
mnist_training_data = Tensor(mnist.train_images()[:2000])
mnist_testing_data = Tensor(mnist.test_images())

# train_mean = mnist_training_data.mean()
# train_std = mnist_training_data.std()

normalized_mnist_training_data = (mnist_training_data) / 255
normalized_mnist_testing_data = (mnist_testing_data) / 255

print('training data shape', mnist_training_data.shape)

# define hyper-parameters
n_X = normalized_mnist_training_data.shape[0]
n_C = int(normalized_mnist_training_data.shape[-2]*normalized_mnist_training_data.shape[-1]) # also data_dim
n_total = n_X * n_C
index_dim = 2
latent_dim = 20
n_inducing_C = 30
n_inducing_X = 50
pca = True
Y_train = normalized_mnist_training_data.reshape(-1)
Y_test  = normalized_mnist_testing_data.reshape(-1)

C = np.empty((28, 28, 2), dtype=int)
for i in range(28):
    for j in range(28):
        C[i, j] = [i, j]

C = Tensor(C.reshape(-1,2))

forbidden_pairs = [(2, 18), (2, 79), (7, 183), (8, 201), (5,23), (6, 99), (0,3), (7, 139), (8, 143), (5, 34)]

# specify model
my_model = LVMOGP_SVI(n_X, n_C, index_dim, latent_dim, n_inducing_C, n_inducing_X, Y_train.reshape(n_X, -1), pca=pca)

# load in partially-trained model
load_model = False
if load_model:
    model_path = '/Users/jiangxiaoyu/Desktop/All Projects/GPLVM_project_code/models/model_weights.pth'
    state_dict = torch.load(model_path)
    my_model.load_state_dict(state_dict)

# Likelihood & training objective
likelihood = GaussianLikelihood() # how many outputs, 
mll = VariationalELBO(likelihood, my_model, num_data=n_total)

# optimizer and scheduler
optimizer = torch.optim.Adam([
    {'params': my_model.parameters()},
    {'params': likelihood.parameters()}
], lr=0.001)

scheduler = StepLR(optimizer, step_size=50, gamma=0.9)  # every 50 iterations，learning rate multiple 0.9

# look at parameters
for name, param in my_model.named_parameters():
    print(name, param.size())

print('---' * 15) 
for name, param in likelihood.named_parameters():
    print(name, param.size())

In [1]:
import numpy as np
from torch import Tensor

C = np.empty((28, 28, 2), dtype=int)
for i in range(28):
    for j in range(28):
        C[i, j] = [i, j]

C = Tensor(C.reshape(-1,2))

from util_functions import proper_gene_forbidden_pairs, tidily_gene_forbidden_pairs
forbidden_list = tidily_gene_forbidden_pairs(n_X=5, n_C=28*28, num_forbidden=3)

In [1]:
from util_functions import *
first_elem_list, second_elem_list = extract_tuple_elements('/Users/jiangxiaoyu/Desktop/All Projects/GPLVM_project_code/experi_results/forbidden_pairs.csv')

In [10]:
import torch
from util_functions import *

C = gene_2dimage_inputs()
print(C.shape)

n_inducing_C = 35
selected_indices = torch.randperm(C.shape[0])[:n_inducing_C]
print(C[selected_indices].shape)

torch.Size([784, 2])
torch.Size([35, 2])
